<a href="https://colab.research.google.com/github/yashguptaab99/Cricket-Prediction/blob/master/Cricket_Predictions_Bowling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INCREASED PREDICTION ACCURACY IN THE GAME OF CRICKET USING MACHINE LEARNING

Player selection is one the most important tasks for any sport and cricket is no exception. The performance of the players depends on various factors such as the opposition team, the venue, his current form etc. The team management, the coach and the captain select 11 players for each match from a squad of 15 to 20 players. They analyze different characteristics and the statistics of the players to select the best playing 11 for each match. Each batsman contributes by scoring maximum runs possible and each bowler contributes by taking maximum wickets and conceding minimum runs. This paper attempts to predict the performance of players as how many runs will each batsman score and how many wickets will each bowler take for both the teams. Both the problems are targeted as classification problems where number of runs and number of wickets are classified in different ranges. We used naïve bayes, random forest, multiclass SVM and decision tree classifiers to generate the prediction models for both the problems. Random Forest classifier was found to be the most accurate for both the problems. 

# Importing Libraray

In [1]:
import pandas as pd
import re

# Importing Data

In [12]:
# All Innings list after 2 Jan 2000
innings = pd.read_csv("/content/drive/My Drive/Projects/Cricket Prediction/Bowling.csv")

# All Ininngs list from 131Mar 1984 to 2 Jan 2000
inningsExtra = pd.read_csv("/content/drive/My Drive/Projects/Cricket Prediction/Bowling84-02.csv")

In [13]:
inningsExtra

,Player,Overs,Mdns,Runs,Wkts,Econ,Inns,Opposition,Ground,Start Date
0,Aamer Hanif,3.4,0,27,1,7.36,1,Sri Lanka,Sharjah,2 Nov 1993
1,Aamer Hanif,10.0,0,38,0,3.80,1,Sri Lanka,Gujranwala,29 Sep 1995
2,Aamer Hanif,2.0,0,21,0,10.50,1,Sri Lanka,Faisalabad,1 Oct 1995
3,Aamer Hanif,6.0,0,36,3,6.00,2,Sri Lanka,Rawalpindi,3 Oct 1995
4,Aamer Hanif,DNB,-,-,-,-,2,Sri Lanka,Sharjah,17 Oct 1995
...,...,...,...,...,...,...,...,...,...,...
28045,Zulqarnain,DNB,-,-,-,-,1,India,Sharjah,18 Apr 1986
28046,Zulqarnain,DNB,-,-,-,-,1,England,Karachi,20 Nov 1987
28047,Zulqarnain,DNB,-,-,-,-,1,England,Peshawar,22 Nov 1987
28048,Zulqarnain,DNB,-,-,-,-,2,India,Gujranwala,18 Dec 1989


# Data Preprocessing

## Bowling data

In [14]:
innings = innings.drop(columns=['Team', 'Mdns', 'Econ', 'Inns'])
inningsExtra = inningsExtra.drop(columns=['Mdns', 'Econ', 'Inns'])

In [15]:
# Cleaning data

innings = innings[innings.Overs != 'DNB']
innings = innings[innings.Overs != 'TDNB']
innings = innings[innings.Overs != 'sub']
innings = innings[innings.Overs != 'absent']
innings = innings.rename(columns={"Player 1":"Player", "Start Date":"StartDate"})

inningsExtra = inningsExtra[inningsExtra.Overs != 'DNB']
inningsExtra = inningsExtra[inningsExtra.Overs != 'TDNB']
inningsExtra = inningsExtra[inningsExtra.Overs != 'sub']
inningsExtra = inningsExtra[inningsExtra.Overs != 'absent']
inningsExtra = inningsExtra.rename(columns={"Start Date":"StartDate"})


In [16]:
innings

,Player,Overs,Runs,Wkts,Opposition,Ground,StartDate
0,NP Kenjige,9,40,1,U.A.E.,Sharjah,8 Dec 2019
1,NP Kenjige,7,50,1,Scotland,Sharjah,9 Dec 2019
2,NP Kenjige,7,26,1,U.A.E.,ICCA Dubai,12 Dec 2019
3,NP Kenjige,3,13,0,Scotland,ICCA Dubai,14 Dec 2019
4,ME Sanuth,6,18,0,U.A.E.,Al Amerat,5 Jan 2020
...,...,...,...,...,...,...,...
60061,Zulfiqar Babar,9,35,1,Australia,Sharjah,7 Oct 2014
60062,Zulfiqar Babar,10,52,2,Australia,Dubai (DSC),10 Oct 2014
60063,Zulfiqar Babar,10,42,0,Australia,Abu Dhabi,12 Oct 2014
60064,Zulfiqar Babar,10,61,1,New Zealand,Abu Dhabi,19 Dec 2014


In [17]:
inningsExtra

,Player,Overs,Runs,Wkts,Opposition,Ground,StartDate
0,Aamer Hanif,3.4,27,1,Sri Lanka,Sharjah,2 Nov 1993
1,Aamer Hanif,10.0,38,0,Sri Lanka,Gujranwala,29 Sep 1995
2,Aamer Hanif,2.0,21,0,Sri Lanka,Faisalabad,1 Oct 1995
3,Aamer Hanif,6.0,36,3,Sri Lanka,Rawalpindi,3 Oct 1995
7,Aamer Malik,2.0,10,0,Sri Lanka,Dhaka,27 Oct 1988
...,...,...,...,...,...,...,...
28024,DNT Zoysa,6.2,24,0,West Indies,Sharjah,13 Oct 1999
28025,DNT Zoysa,8.0,24,0,Pakistan,Sharjah,15 Oct 1999
28026,DNT Zoysa,7.0,14,1,West Indies,Sharjah,17 Oct 1999
28027,DNT Zoysa,8.0,32,1,Pakistan,Sharjah,18 Oct 1999


In [19]:
#List of all players who played after 2 Jan 2000

listOfBowler = list(innings['Player'].unique())

In [20]:
#Merging player past performance innings which were present in matches after 2005
# for ex. Sachine was senior most so his mast matches performance shoould be added 

for player in listOfBowler:
  playerframe = inningsExtra[inningsExtra.Player == player]
  innings = innings.append(playerframe)


In [21]:
innings['StartDate'] = pd.to_datetime(innings['StartDate'])
# Now innings variable contains all players past played innings
innings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39927 entries, 0 to 28028
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Player      39927 non-null  object        
 1   Overs       39927 non-null  object        
 2   Runs        39927 non-null  object        
 3   Wkts        39927 non-null  object        
 4   Opposition  39927 non-null  object        
 5   Ground      39927 non-null  object        
 6   StartDate   39927 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(6)
memory usage: 2.4+ MB


In [23]:
#Converting Over to integer and cleaning it
Over = []
for st in innings['Overs'].values:
  st = re.findall(r'[0-9]+', st)
  if not st:
    st.append('0')
  Over.append(float(st[0]))
innings['Overs'] = Over

In [24]:
#Converting Run to integer and cleaning it
run = []
for st in innings['Runs'].values:
  st = re.findall(r'[0-9]+', st)
  if not st:
    st.append('0')
  run.append(float(st[0]))
innings['Runs'] = run

In [26]:
#Converting Run to integer and cleaning it
wkt = []
for st in innings['Wkts'].values:
  st = re.findall(r'[0-9]+', st)
  if not st:
    st.append('0')
  wkt.append(float(st[0]))
innings['Wkts'] = wkt

In [27]:
innings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39927 entries, 0 to 28028
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Player      39927 non-null  object        
 1   Overs       39927 non-null  float64       
 2   Runs        39927 non-null  float64       
 3   Wkts        39927 non-null  float64       
 4   Opposition  39927 non-null  object        
 5   Ground      39927 non-null  object        
 6   StartDate   39927 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 2.4+ MB


### Calculating The Derived Attributes

#### Consistency

This attribute describes how experienced the player is and how consistent he has been throughout his career. All the traditional attributes used in this formula are calculated over the entire career of the player. 

**Consistency = 0.4174Xno. of overs + 0.2634Xno. of innings + 0.1602XSR + 0.0975Xaverage + 0.0615XFF**


In [28]:
#Please Rate then Calculate

## Consistency 
Consistency = []

for player in listOfBowler:
  runs_conceede = 0
  balls_bowled = 0
  wkt_taken = 0
  playerframe = innings[innings.Player == player]

  ######### Number of innings #########
  numInnings = playerframe.shape[0]

  ######### Overs #########
  overs = playerframe['Overs'].sum()

  ######### Runs concede #########
  runs_concede = playerframe['Runs'].sum()

  ######### Wicket Taken #########
  wkt_taken = playerframe['Wkts'].sum()

  ######### Total Ball bowled #########
  balls_bowled = 6 * overs

  ######### Bowling Average #########
  if (wkt_taken==0):
    average = 0
  else:
    average = runs_concede/wkt_taken

  ######### Strike Rate #########
  if (wkt_taken==0):
    sr = 0
  else:
    sr = (balls_bowled/wkt_taken) * 100

  ######### Wickets haul #########
  ff = playerframe[playerframe.Wkts > 4].shape[0]

####################  Rate the Elements Before Calculation  ####################

  #### numInnings ####
  if (numInnings>=1 and numInnings<=49):
    numInnings = 1
  elif (numInnings>=50 and numInnings<=99):
    numInnings = 2
  elif (numInnings>=100 and numInnings<=124):
    numInnings = 3
  elif (numInnings>=125 and numInnings<=149):
    numInnings = 4
  elif (numInnings>=150):
    numInnings = 5 

  #### average ####
  if (average>=0.0 and average<=24.9):
    average = 1
  elif (average>=25.0 and average<=29.9):
    average = 2
  elif (average>=30.0 and average<=34.9):
    average = 3
  elif (average>=35.0 and average<=49.9):
    average = 4
  elif (average>=50.0):
    average = 5   

  #### sr ####
  if (sr>=0.0 and sr<=29.9):
    sr = 1
  elif (sr>=30.0 and sr<=39.9):
    sr = 2
  elif (sr>=40.0 and sr<=49.0):
    sr = 3
  elif (sr>=50.0 and sr<=59.9):
    sr = 4
  elif (sr>=60.0):
    sr = 5  

  #### Overs ####
  if (overs>=1 and overs<=99):
    overs = 1
  elif (overs>=100 and overs<=249):
    overs = 2
  elif (overs>=250 and overs<=499):
    overs = 3
  elif (overs>=500 and overs<=999):
    overs = 4
  elif (overs>=1000):
    overs = 5 

  #### ff ####
  if (ff>=1 and ff<=2):
    ff = 3
  elif (ff>=3 and ff<=4):
    ff = 4
  elif (ff>=5):
    ff = 5

  consistency =  (0.4174 * overs) + (0.2634 * numInnings) + (0.1602 * sr) + (0.0975*average) + (0.0615 * ff) 
  Consistency.append(consistency)


In [29]:
ConsistencyFrame = pd.DataFrame(Consistency, columns = ["Consistency"])

In [30]:
ConsistencyFrame

,Consistency
0,1.6768
1,0.9385
2,3.9142
3,1.9693
4,1.6768
...,...
1307,0.9385
1308,1.9588
1309,1.8718
1310,3.3769


#### Form

Form of a player describes his performance over last one year. All the traditional attributes used in this formula are calculated over the matches played by the player in last 12 months from the day of the match. 

**Form = (0.3269 * overs) + (0.2846 * numInnings) + (0.1877 * sr) + (0.1210 * average) + (0.0798 * ff)**

In [44]:
#Please Rate then Calculate

## Form
Form = []

for player in listOfBowler:
  playerframe = innings[innings.Player == player]
  playerframe = playerframe[playerframe.StartDate > "2019-01-01"]

  if not playerframe.empty:
    runs_conceede = 0
    balls_bowled = 0
    wkt_taken = 0
    playerframe = innings[innings.Player == player]

    ######### Number of innings #########
    numInnings = playerframe.shape[0]

    ######### Overs #########
    overs = playerframe['Overs'].sum()

    ######### Runs concede #########
    runs_concede = playerframe['Runs'].sum()

    ######### Wicket Taken #########
    wkt_taken = playerframe['Wkts'].sum()

    ######### Total Ball bowled #########
    balls_bowled = 6 * overs

    ######### Bowling Average #########
    if (wkt_taken==0):
      average = 0
    else:
      average = runs_concede/wkt_taken

    ######### Strike Rate #########
    if (wkt_taken==0):
      sr = 0
    else:
      sr = (balls_bowled/wkt_taken) * 100

    ######### Wickets haul #########
    ff = playerframe[playerframe.Wkts > 4].shape[0]

  ####################  Rate the Elements Before Calculation  ####################

    #### numInnings ####
    if (numInnings>=1 and numInnings<=4):
      numInnings = 1
    elif (numInnings>=5 and numInnings<=9):
      numInnings = 2
    elif (numInnings>=10 and numInnings<=13):
      numInnings = 3
    elif (numInnings>=12 and numInnings<=14):
      numInnings = 4
    elif (numInnings>=15):
      numInnings = 5 

    #### average ####
    if (average>=0.0 and average<=24.9):
      average = 1
    elif (average>=25.0 and average<=29.9):
      average = 2
    elif (average>=30.0 and average<=34.9):
      average = 3
    elif (average>=35.0 and average<=49.9):
      average = 4
    elif (average>=50.0):
      average = 5   

    #### sr ####
    if (sr>=0.0 and sr<=29.9):
      sr = 1
    elif (sr>=30.0 and sr<=39.9):
      sr = 2
    elif (sr>=40.0 and sr<=49.0):
      sr = 3
    elif (sr>=50.0 and sr<=59.9):
      sr = 4
    elif (sr>=60.0):
      sr = 5  

    #### Overs ####
    if (overs>=1 and overs<=9):
      overs = 1
    elif (overs>=10 and overs<=24):
      overs = 2
    elif (overs>=25 and overs<=49):
      overs = 3
    elif (overs>=50 and overs<=99):
      overs = 4
    elif (overs>=100):
      overs = 5 

    #### ff ####
    if (ff>=1 and ff<=2):
      ff = 4
    elif (ff>=3):
      ff = 5

    form =  (0.3269 * overs) + (0.2846 * numInnings) + (0.1877 * sr) + (0.1210 * average) + (0.0798 * ff)
  else:
    form = 0

  Form.append(form)

In [45]:
FormFrame = pd.DataFrame(Form, columns=['Form'])

In [46]:
FormFrame

,Form
0,2.7304
1,1.2471
2,0.0000
3,0.0000
4,2.1189
...,...
1307,0.0000
1308,0.0000
1309,0.0000
1310,0.0000


#### Opposition

Opposition describes a player’s performance against a particular team. All the traditional attributes used in this formula are calculated over all the matches played by the player against the opposition team in his entire career till the day of the match. 

**Opposition = (0.3177 * overs) + (0.3177 * numInnings) + (0.1933 * sr) + (0.1465 * average) + (0.0943 * ff)** 

In [47]:
listOfOpposition = list(innings['Opposition'].unique())

In [49]:
#Please Rate then Calculate

## Opposition 
Oppositions = []

for player in listOfBowler:
  playerframe = innings[innings.Player == player]
  perPlayerOpposition = []
  for opposition in listOfOpposition:
    oppositionframe = playerframe[playerframe.Opposition == opposition]
    if not oppositionframe.empty:
      runs_conceede = 0
      balls_bowled = 0
      wkt_taken = 0
      playerframe = innings[innings.Player == player]

      ######### Number of innings #########
      numInnings = playerframe.shape[0]

      ######### Overs #########
      overs = playerframe['Overs'].sum()

      ######### Runs concede #########
      runs_concede = playerframe['Runs'].sum()

      ######### Wicket Taken #########
      wkt_taken = playerframe['Wkts'].sum()

      ######### Total Ball bowled #########
      balls_bowled = 6 * overs

      ######### Bowling Average #########
      if (wkt_taken==0):
        average = 0
      else:
        average = runs_concede/wkt_taken

      ######### Strike Rate #########
      if (wkt_taken==0):
        sr = 0
      else:
        sr = (balls_bowled/wkt_taken) * 100

      ######### Wickets haul #########
      ff = playerframe[playerframe.Wkts > 4].shape[0]

    ####################  Rate the Elements Before Calculation  ####################

      #### numInnings ####
      if (numInnings>=1 and numInnings<=2):
        numInnings = 1
      elif (numInnings>=3 and numInnings<=4):
        numInnings = 2
      elif (numInnings>=5 and numInnings<=6):
        numInnings = 3
      elif (numInnings>=7 and numInnings<=9):
        numInnings = 4
      elif (numInnings>=10):
        numInnings = 5 

      #### average ####
      if (average>=0.0 and average<=24.9):
        average = 1
      elif (average>=25.0 and average<=29.9):
        average = 2
      elif (average>=30.0 and average<=34.9):
        average = 3
      elif (average>=35.0 and average<=49.9):
        average = 4
      elif (average>=50.0):
        average = 5   

      #### sr ####
      if (sr>=0.0 and sr<=29.9):
        sr = 1
      elif (sr>=30.0 and sr<=39.9):
        sr = 2
      elif (sr>=40.0 and sr<=49.0):
        sr = 3
      elif (sr>=50.0 and sr<=59.9):
        sr = 4
      elif (sr>=60.0):
        sr = 5  

      #### Overs ####
      if (overs>=1 and overs<=9):
        overs = 1
      elif (overs>=10 and overs<=24):
        overs = 2
      elif (overs>=25 and overs<=49):
        overs = 3
      elif (overs>=50 and overs<=99):
        overs = 4
      elif (overs>=100):
        overs = 5 

      #### ff ####
      if (ff>=1 and ff<=2):
        ff = 4
      elif (ff>=3):
        ff = 5

      oppo =  (0.3177 * overs) + (0.3177 * numInnings) + (0.1933 * sr) + (0.1465 * average) + (0.0943 * ff)
    else:
      oppo = 0
    perPlayerOpposition.append(oppo)
  Oppositions.append(perPlayerOpposition)   

In [50]:
OppositionsFrame = pd.DataFrame(Oppositions, columns = listOfOpposition) 

In [51]:
OppositionsFrame

,U.A.E.,Scotland,Namibia,Nepal,Sri Lanka,Zimbabwe,New Zealand,England,West Indies,Bangladesh,Pakistan,Australia,Ireland,India,Kenya,South Africa,Netherlands,Hong Kong,ICC World XI,Africa XI,Canada,Bermuda,Afghanistan,P.N.G.,U.S.A.,Oman,Asia XI
0,3.1657,3.1657,0.0000,3.1657,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000
1,1.6106,0.0000,1.6106,1.6106,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000
2,4.7295,0.0000,0.0000,0.0000,4.7295,4.7295,4.7295,4.7295,4.7295,4.7295,0.0000,4.7295,0.0000,4.7295,0.0000,4.7295,4.7295,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000
3,0.0000,0.0000,0.0000,0.0000,0.0000,3.2875,3.2875,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000
4,0.0000,2.2126,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.9752,0.0000,0.0000,0.9752,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000
1308,0.0000,0.0000,0.0000,0.0000,4.6425,4.6425,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,4.6425,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,4.6425,0.0,0.0,0.0,0.0000
1309,0.0000,0.0000,0.0000,0.0000,4.4118,4.4118,0.0000,0.0000,0.0000,4.4118,4.4118,4.4118,0.0000,0.0000,4.4118,0.0000,0.0000,0.0,0.0,0.0,4.4118,0.0,0.0000,0.0,0.0,0.0,4.4118
1310,4.8137,0.0000,0.0000,0.0000,0.0000,4.8137,4.8137,4.8137,4.8137,4.8137,4.8137,4.8137,0.0000,4.8137,0.0000,4.8137,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000


In [52]:
OppositionsFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1312 entries, 0 to 1311
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   U.A.E.        1312 non-null   float64
 1   Scotland      1312 non-null   float64
 2   Namibia       1312 non-null   float64
 3   Nepal         1312 non-null   float64
 4   Sri Lanka     1312 non-null   float64
 5   Zimbabwe      1312 non-null   float64
 6   New Zealand   1312 non-null   float64
 7   England       1312 non-null   float64
 8   West Indies   1312 non-null   float64
 9   Bangladesh    1312 non-null   float64
 10  Pakistan      1312 non-null   float64
 11  Australia     1312 non-null   float64
 12  Ireland       1312 non-null   float64
 13  India         1312 non-null   float64
 14  Kenya         1312 non-null   float64
 15  South Africa  1312 non-null   float64
 16  Netherlands   1312 non-null   float64
 17  Hong Kong     1312 non-null   float64
 18  ICC World XI  1312 non-null 

#### Venue

Venue describes a player’s performance at a particular venue. All the traditional attributes used in this formula are calculated over all the matches played by the player at the venue in his entire career till the day of the match. 

**Venue = (0.3018 * overs) + (0.2783 * numInnings) + (0.1836 * sr) + (0.1391 * average) + (0.0972 * ff)**

In [53]:
listOfVenue = list(innings['Ground'].unique())

In [54]:
#Please Rate then Calculate

## Venue
Venues = []

for player in listOfBowler:
  playerframe = innings[innings.Player == player]
  perPlayerVenue = []
  for venue in listOfVenue:
    venueframe = playerframe[playerframe.Ground == venue]
    if not venueframe.empty:
      runs_conceede = 0
      balls_bowled = 0
      wkt_taken = 0
      playerframe = innings[innings.Player == player]

      ######### Number of innings #########
      numInnings = playerframe.shape[0]

      ######### Overs #########
      overs = playerframe['Overs'].sum()

      ######### Runs concede #########
      runs_concede = playerframe['Runs'].sum()

      ######### Wicket Taken #########
      wkt_taken = playerframe['Wkts'].sum()

      ######### Total Ball bowled #########
      balls_bowled = 6 * overs

      ######### Bowling Average #########
      if (wkt_taken==0):
        average = 0
      else:
        average = runs_concede/wkt_taken

      ######### Strike Rate #########
      if (wkt_taken==0):
        sr = 0
      else:
        sr = (balls_bowled/wkt_taken) * 100

      ######### Wickets haul #########
      ff = playerframe[playerframe.Wkts > 4].shape[0]

    ####################  Rate the Elements Before Calculation  ####################

    #### numInnings ####
      if (numInnings==1):
        numInnings = 1
      elif (numInnings==2):
        numInnings = 2
      elif (numInnings==3):
        numInnings = 3
      elif (numInnings==4):
        numInnings = 4
      elif (numInnings>=5):
        numInnings = 5 

      #### average ####
      if (average>=0.0 and average<=24.9):
        average = 1
      elif (average>=25.0 and average<=29.9):
        average = 2
      elif (average>=30.0 and average<=34.9):
        average = 3
      elif (average>=35.0 and average<=49.9):
        average = 4
      elif (average>=50.0):
        average = 5   

      #### sr ####
      if (sr>=0.0 and sr<=29.9):
        sr = 1
      elif (sr>=30.0 and sr<=39.9):
        sr = 2
      elif (sr>=40.0 and sr<=49.0):
        sr = 3
      elif (sr>=50.0 and sr<=59.9):
        sr = 4
      elif (sr>=60.0):
        sr = 5  

      #### Overs ####
      if (overs>=1 and overs<=9):
        overs = 1
      elif (overs>=10 and overs<=19):
        overs = 2
      elif (overs>=20 and overs<=29):
        overs = 3
      elif (overs>=30 and overs<=39):
        overs = 4
      elif (overs>=40):
        overs = 5 

      #### ff ####
      if (ff>=1 and ff<=2):
        ff = 4
      elif (ff>=3):
        ff = 5

      ven = (0.3018 * overs) + (0.2783 * numInnings) + (0.1836 * sr) + (0.1391 * average) + (0.0972 * ff) 
    else:
      ven = 0
    perPlayerVenue.append(ven)
  Venues.append(perPlayerVenue)    

In [55]:
VenuesFrame = pd.DataFrame(Venues, columns = listOfVenue)

In [56]:
VenuesFrame

,Sharjah,ICCA Dubai,Al Amerat,Kirtipur,Karachi,Gujranwala,Lahore,Harare,Wellington,Aberdeen,Mumbai,Kolkata,Cuttack,Visakhapatnam,Auckland,Hamilton,Fatullah,Nairobi,Bulawayo,Nairobi (Gym),Bloemfontein,Centurion,Melbourne,Sydney,Port Elizabeth,Canberra,Dhaka,Chattogram,Durban,Johannesburg,Cape Town,Providence,Basseterre,Brisbane,Hobart,Adelaide,St John's,Kingstown,St George's,Bridgetown,...,Greater Noida,Nelson,Kwekwe,Kandy,Ayr,Ranchi,Dharamsala,Whangarei,Melbourne (Docklands),Schiedam,Toronto,Deventer,Georgetown,Nairobi (Jaff),Cairns,Chandigarh,Hyderabad (Sind),Lucknow,Port Moresby,Quetta,Derby,Hove,Leicester,Chelmsford,Jalandhar,Berri,Amritsar,Sialkot,Northampton,Worcester,Nairobi (Club),Moratuwa,New Delhi,Nairobi (Aga),Sargodha,New Plymouth,Ballarat,Patna,Devonport,Srinagar
0,3.4931,3.4931,0.0000,3.4931,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0000,0.0000,1.7612,1.7612,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.3749,0.0000,0.0000,0.0000,4.3749,4.3749,4.3749,4.3749,4.3749,0.0000,0.0,0.0,0.0,0.0,4.3749,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,4.3749,4.3749,4.3749,4.3749,0.0,4.3749,0.0000,4.3749,4.3749,4.3749,0.0,0.0,4.3749,4.3749,4.3749,0.0,4.3749,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,4.3749,0.0,4.3749,0.0,0.0,0.0,0.0,0.0,0.0,4.3749,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,3.6321,3.6321,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2.3564,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0000,1.1811,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,1.1811,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1308,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0000,0.0000,4.6246,0.0,4.6246,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,4.6246,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,

In [57]:
VenuesFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1312 entries, 0 to 1311
Columns: 172 entries, Sharjah to Srinagar
dtypes: float64(172)
memory usage: 1.7 MB


In [58]:
listOfBowlerFrame = pd.DataFrame(listOfBowler, columns=["Players"])

In [59]:
listOfBowlerFrame

,Players
0,NP Kenjige
1,ME Sanuth
2,Aamer Sohail
3,Aamer Yamin
4,Aamir Kaleem
...,...
1307,C Zhuwao
1308,Ziaur Rahman
1309,M Zondeki
1310,DNT Zoysa


#### Final Data for training the model

In [60]:
playerPerformance = pd.concat([listOfBowlerFrame, ConsistencyFrame, FormFrame, OppositionsFrame, VenuesFrame], axis = 1)

In [61]:
playerPerformance

,Players,Consistency,Form,U.A.E.,Scotland,Namibia,Nepal,Sri Lanka,Zimbabwe,New Zealand,England,West Indies,Bangladesh,Pakistan,Australia,Ireland,India,Kenya,South Africa,Netherlands,Hong Kong,ICC World XI,Africa XI,Canada,Bermuda,Afghanistan,P.N.G.,U.S.A.,Oman,Asia XI,Sharjah,ICCA Dubai,Al Amerat,Kirtipur,Karachi,Gujranwala,Lahore,Harare,Wellington,Aberdeen,...,Greater Noida,Nelson,Kwekwe,Kandy,Ayr,Ranchi,Dharamsala,Whangarei,Melbourne (Docklands),Schiedam,Toronto,Deventer,Georgetown,Nairobi (Jaff),Cairns,Chandigarh,Hyderabad (Sind),Lucknow,Port Moresby,Quetta,Derby,Hove,Leicester,Chelmsford,Jalandhar,Berri,Amritsar,Sialkot,Northampton,Worcester,Nairobi (Club),Moratuwa,New Delhi,Nairobi (Aga),Sargodha,New Plymouth,Ballarat,Patna,Devonport,Srinagar
0,NP Kenjige,1.6768,2.7304,3.1657,3.1657,0.0000,3.1657,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,3.4931,3.4931,0.0000,3.4931,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ME Sanuth,0.9385,1.2471,1.6106,0.0000,1.6106,1.6106,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,1.7612,1.7612,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aamer Sohail,3.9142,0.0000,4.7295,0.0000,0.0000,0.0000,4.7295,4.7295,4.7295,4.7295,4.7295,4.7295,0.0000,4.7295,0.0000,4.7295,0.0000,4.7295,4.7295,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,4.3749,0.0000,0.0000,0.0000,4.3749,4.3749,4.3749,4.3749,4.3749,0.0000,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,4.3749,0.0,4.3749,0.0,0.0,0.0,0.0,0.0,0.0,4.3749,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aamer Yamin,1.9693,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,3.2875,3.2875,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,3.6321,3.6321,0.0000,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aamir Kaleem,1.6768,2.1189,0.0000,2.2126,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2.3564,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,C Zhuwao,0.9385,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.9752,0.0000,0.0000,0.9752,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1308,Ziaur Rahman,1.9588,0.0000,0.0000,0.0000,0.0000,0.0000,4.6425,4.6425,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,4.6425,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0000,0.0,4.6425,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,

In [63]:
bowling = pd.read_csv("/content/drive/My Drive/Projects/Cricket Prediction/Bowling.csv")

In [64]:
# Cleaning data

bowling = bowling[bowling.Runs != 'DNB']
bowling = bowling[bowling.Runs != 'TDNB']
bowling = bowling[bowling.Runs != 'sub']
bowling = bowling[bowling.Runs != 'absent']
bowling = bowling.rename(columns={"Player 1":"Players", "Start Date":"StartDate"})

In [65]:
bowling = bowling.drop(columns=['Team', 'Mdns', 'Econ', 'Inns', 'Overs','Runs', 'Opposition', 'Ground', 'StartDate'])

In [66]:
bowling

,Players,Wkts
0,NP Kenjige,1
1,NP Kenjige,1
2,NP Kenjige,1
3,NP Kenjige,0
4,ME Sanuth,0
...,...,...
60065,Zulfiqar Babar,0
60066,Zulqarnain Haider,-
60067,Zulqarnain Haider,-
60068,Zulqarnain Haider,-


In [67]:
wkt = []
for st in bowling['Wkts'].values:
  st = re.findall(r'[0-9]+', st)
  if not st:
    st.append('0')
  r = float(st[0])
  ######## Rate the run attribute ########
  if (r>=0 and r<=1):
    r = 1
  elif (r>=2 and r<=3):
    r = 2
  elif (r>=4):
    r = 3  
  wkt.append(r)
bowling['Wkts'] = wkt

In [68]:
bowling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60070 entries, 0 to 60069
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Players  60070 non-null  object
 1   Wkts     60070 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ MB


In [69]:
bowling

,Players,Wkts
0,NP Kenjige,1
1,NP Kenjige,1
2,NP Kenjige,1
3,NP Kenjige,1
4,ME Sanuth,1
...,...,...
60065,Zulfiqar Babar,1
60066,Zulqarnain Haider,1
60067,Zulqarnain Haider,1
60068,Zulqarnain Haider,1


In [70]:
# Now we have to join playerPerformance and bowling dataframe to create our final dataset

finalbowling = pd.merge(bowling, playerPerformance, on="Players")

In [71]:
finalbowling.isna().any()

Players         False
Wkts            False
Consistency     False
Form            False
U.A.E.          False
                ...  
New Plymouth    False
Ballarat        False
Patna           False
Devonport       False
Srinagar        False
Length: 203, dtype: bool

In [72]:
finalbowling.to_csv("/content/drive/My Drive/Projects/Cricket Prediction/finalbowling.csv")

In [73]:
finalbowling = pd.read_csv("/content/drive/My Drive/Projects/Cricket Prediction/finalbowling.csv")

In [76]:
X_bowling = finalbowling.drop(columns=['Wkts', 'Unnamed: 0', 'Players'])
y_bowling = finalbowling['Wkts']

In [77]:
X_bowling

,Consistency,Form,U.A.E.,Scotland,Namibia,Nepal,Sri Lanka,Zimbabwe,New Zealand,England,West Indies,Bangladesh,Pakistan,Australia,Ireland,India,Kenya,South Africa,Netherlands,Hong Kong,ICC World XI,Africa XI,Canada,Bermuda,Afghanistan,P.N.G.,U.S.A.,Oman,Asia XI,Sharjah,ICCA Dubai,Al Amerat,Kirtipur,Karachi,Gujranwala,Lahore,Harare,Wellington,Aberdeen,Mumbai,...,Greater Noida,Nelson,Kwekwe,Kandy,Ayr,Ranchi,Dharamsala,Whangarei,Melbourne (Docklands),Schiedam,Toronto,Deventer,Georgetown,Nairobi (Jaff),Cairns,Chandigarh,Hyderabad (Sind),Lucknow,Port Moresby,Quetta,Derby,Hove,Leicester,Chelmsford,Jalandhar,Berri,Amritsar,Sialkot,Northampton,Worcester,Nairobi (Club),Moratuwa,New Delhi,Nairobi (Aga),Sargodha,New Plymouth,Ballarat,Patna,Devonport,Srinagar
0,1.6768,2.7304,3.1657,3.1657,0.0,3.1657,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.4931,3.4931,0.0,3.4931,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.6768,2.7304,3.1657,3.1657,0.0,3.1657,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.4931,3.4931,0.0,3.4931,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.6768,2.7304,3.1657,3.1657,0.0,3.1657,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.4931,3.4931,0.0,3.4931,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.6768,2.7304,3.1657,3.1657,0.0,3.1657,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.4931,3.4931,0.0,3.4931,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.6768,2.7304,3.1657,3.1657,0.0,3.1657,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.4931,3.4931,0.0,3.4931,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48469,1.9693,0.0000,0.0000,0.0000,0.0,0.0000,0.0,0.0,3.6052,0.0,0.0,3.6052,0.0,3.6052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5140,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48470,1.9693,0.0000,0.0000,0.0000,0.0,0.0000,0.0,0.0,3.6052,0.0,0.0,3.6052,0.0,3.6052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5140,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48471,1.9693,0.0000,0.0000,0.0000,0.0,0.0000,0.0,0.0,3.6052,0.0,0.0,3.6052,0.0,3.6052,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5140,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48472,1.9693,0.0000,0.0000,0.00

In [78]:
y_bowling

0        1
1        1
2        1
3        1
4        2
        ..
48469    1
48470    2
48471    1
48472    1
48473    1
Name: Wkts, Length: 48474, dtype: int64

# Oversampling SMOTE

In [79]:
pip install -U imbalanced-learn

     |████████████████████████████████| 174kB 4.6MB/s 
     |████████████████████████████████| 6.9MB 9.1MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Found existing installation: imbalanced-learn 0.4.3
    Uninstalling imbalanced-learn-0.4.3:
      Successfully uninstalled imbalanced-learn-0.4.3


In [80]:
from imblearn.over_sampling import SMOTE
X_resample_bowling, y_resample_bowling = SMOTE().fit_sample(X_bowling,y_bowling.values.ravel())

In [81]:
X_resample_bowling.shape

(115260, 201)

In [82]:
X_bowling.shape

(48474, 201)

# Splitting the datasets into the Training set and Test set

In [83]:
###############   FOR bowling   ###############
from sklearn.model_selection import train_test_split
X_train_bowling, X_test_bowling, y_train_bowling, y_test_bowling = train_test_split(X_resample_bowling, y_resample_bowling, test_size=0.3, random_state = 1)

In [84]:
print(X_train_bowling.shape)

print(X_test_bowling.shape)

print(y_train_bowling.shape)

print(y_test_bowling.shape)

(80682, 201)
(34578, 201)
(80682,)
(34578,)


In [85]:
X_train_bowling

,Consistency,Form,U.A.E.,Scotland,Namibia,Nepal,Sri Lanka,Zimbabwe,New Zealand,England,West Indies,Bangladesh,Pakistan,Australia,Ireland,India,Kenya,South Africa,Netherlands,Hong Kong,ICC World XI,Africa XI,Canada,Bermuda,Afghanistan,P.N.G.,U.S.A.,Oman,Asia XI,Sharjah,ICCA Dubai,Al Amerat,Kirtipur,Karachi,Gujranwala,Lahore,Harare,Wellington,Aberdeen,Mumbai,...,Greater Noida,Nelson,Kwekwe,Kandy,Ayr,Ranchi,Dharamsala,Whangarei,Melbourne (Docklands),Schiedam,Toronto,Deventer,Georgetown,Nairobi (Jaff),Cairns,Chandigarh,Hyderabad (Sind),Lucknow,Port Moresby,Quetta,Derby,Hove,Leicester,Chelmsford,Jalandhar,Berri,Amritsar,Sialkot,Northampton,Worcester,Nairobi (Club),Moratuwa,New Delhi,Nairobi (Aga),Sargodha,New Plymouth,Ballarat,Patna,Devonport,Srinagar
40987,4.584500,0.000000,0.000000,0.000000,4.8137,0.0,4.813700,0.000000,4.813700,4.813700,4.81370,4.813700,4.813700,4.813700,0.000000,4.813700,4.813700,4.813700,4.813700,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,4.8137,4.485500,0.0,0.0,0.0,4.4855,4.4855,4.4855,4.4855,4.4855,0.000000,0.0000,...,0.000000,0.0,0.0,4.4855,0.000000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.4855,4.4855,0.0,0.0,0.0,0.0,0.0,4.4855,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.4855,0.0,0.0
357,4.646000,0.000000,4.908000,4.908000,0.0000,0.0,4.908000,4.908000,4.908000,4.908000,4.90800,0.000000,4.908000,4.908000,4.908000,4.908000,4.908000,4.908000,4.908000,4.908000,0.0,0.0,4.908000,4.908000,4.908000,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,4.5827,0.0000,4.5827,4.5827,0.0000,0.000000,0.0000,...,0.000000,0.0,0.0,0.0000,0.000000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
44691,4.397936,0.000000,0.000000,8.207482,0.0000,0.0,0.000000,8.207482,0.000000,8.207482,0.00000,0.000000,8.207482,8.207482,8.207482,0.000000,8.207482,8.207482,0.000000,0.000000,0.0,0.0,8.207482,8.207482,8.207482,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.000000,0.0000,...,0.000000,0.0,0.0,0.0000,0.000000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
98715,4.155200,0.000000,0.000000,4.960200,0.0000,0.0,4.960200,0.000000,4.960200,0.000000,4.96020,4.960200,4.960200,4.960200,4.960200,4.960200,0.000000,4.960200,4.960200,0.000000,0.0,0.0,0.000000,0.000000,4.960200,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.0000,0.0000,0.0000,0.0000,4.6246,0.000000,0.0000,...,0.000000,0.0,0.0,0.0000,0.000000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
76064,2.598600,0.000000,0.000000,0.000000,0.0000,0.0,4.667200,4.667200,4.667200,0.000000,4.66720,4.667200,4.667200,4.667200,0.000000,0.000000,0.000000,4.667200,0.000000,0.000000,0.0,0.0,0.000000,0.000000,4.667200,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,0.0000,0.0000,0.0000,4.3464,0.0000,0.000000,4.3464,...,0.000000,0.0,0.0,0.0000,0.000000,4.3464,4.3464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50057,4.584500,0.000000,0.000000,4.813700,4.8137,0.0,4.813700,4.813700,4.813700,0.000000,4.81370,4.813700,4.813700,4.813700,4.813700,4.813700,4.813700,4.813700,4.813700,0.000000,0.0,0.0,4.813700,0.000000,4.813700,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0,0.0,4.4855,0.0000,4.4855,4.4855,4.4855,4.485500,0.0000,...,0.000000,0.0,0.0,0.0000,0.000000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,

# Feature Scaleing

In [86]:
###############   FOR bowling   ###############
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_bowling = pd.DataFrame(sc.fit_transform(X_train_bowling), columns = X_train_bowling.columns.values, index = X_train_bowling.index)
X_test_bowling = pd.DataFrame(sc.transform(X_test_bowling), columns = X_test_bowling.columns.values, index = X_test_bowling.index)

In [87]:
X_train_bowling

,Consistency,Form,U.A.E.,Scotland,Namibia,Nepal,Sri Lanka,Zimbabwe,New Zealand,England,West Indies,Bangladesh,Pakistan,Australia,Ireland,India,Kenya,South Africa,Netherlands,Hong Kong,ICC World XI,Africa XI,Canada,Bermuda,Afghanistan,P.N.G.,U.S.A.,Oman,Asia XI,Sharjah,ICCA Dubai,Al Amerat,Kirtipur,Karachi,Gujranwala,Lahore,Harare,Wellington,Aberdeen,Mumbai,...,Greater Noida,Nelson,Kwekwe,Kandy,Ayr,Ranchi,Dharamsala,Whangarei,Melbourne (Docklands),Schiedam,Toronto,Deventer,Georgetown,Nairobi (Jaff),Cairns,Chandigarh,Hyderabad (Sind),Lucknow,Port Moresby,Quetta,Derby,Hove,Leicester,Chelmsford,Jalandhar,Berri,Amritsar,Sialkot,Northampton,Worcester,Nairobi (Club),Moratuwa,New Delhi,Nairobi (Aga),Sargodha,New Plymouth,Ballarat,Patna,Devonport,Srinagar
40987,1.212604,-0.643452,-0.592187,-0.861957,3.118605,-0.103276,0.550027,-1.519691,0.593026,0.595891,0.550277,0.553586,0.565943,0.600964,-1.111364,0.582127,1.103286,0.606723,1.313479,-0.368655,-0.261265,-0.280848,-0.676825,-0.387994,-0.750480,-0.224626,-0.243328,-0.141667,3.850088,1.017415,-0.265305,-0.082683,-0.085964,1.528206,3.560512,1.401583,0.804073,1.092821,-0.285522,-0.630555,...,-0.165527,-0.451885,-0.123157,4.363981,-0.197695,-0.321001,-0.308827,-0.210567,-0.440283,-0.104728,-0.338442,-0.097507,-0.26872,-0.19559,-0.189446,-0.242677,-0.226082,-0.167025,-0.101344,-0.149435,-0.16783,-0.182647,6.803184,5.340963,-0.175678,-0.032054,-0.106901,-0.159448,-0.257318,4.995603,-0.147242,-0.152215,-0.082305,-0.165547,-0.097026,-0.061946,-0.061946,6.754732,-0.004979,-0.021419
357,1.267139,-0.643452,1.781551,1.174687,-0.308540,-0.103276,0.594901,0.693156,0.635762,0.638947,0.595055,-1.695187,0.610411,0.643479,0.929785,0.625243,1.142286,0.649492,1.353486,2.820548,-0.261265,-0.280848,1.542983,2.687330,1.388970,-0.224626,-0.243328,-0.141667,-0.263995,-0.948090,-0.265305,-0.082683,-0.085964,1.575122,-0.251601,1.446836,0.847947,-0.872972,-0.285522,-0.630555,...,-0.165527,-0.451885,-0.123157,-0.232351,-0.197695,-0.321001,-0.308827,-0.210567,-0.440283,-0.104728,-0.338442,-0.097507,-0.26872,-0.19559,-0.189446,-0.242677,-0.226082,-0.167025,-0.101344,-0.149435,-0.16783,-0.182647,-0.148962,-0.171994,-0.175678,-0.032054,-0.106901,-0.159448,-0.257318,-0.200138,-0.147242,-0.152215,-0.082305,-0.165547,-0.097026,-0.061946,-0.061946,-0.151059,-0.004979,-0.021419
44691,1.047170,-0.643452,-0.592187,2.543854,-0.308540,-0.103276,-1.740603,2.180777,-1.588509,2.145437,-1.735502,-1.695187,2.166335,2.131053,2.301981,-1.618767,2.506884,2.145951,-0.728760,-0.368655,-0.261265,-0.280848,3.035284,4.754766,2.827249,-0.224626,-0.243328,-0.141667,-0.263995,-0.948090,-0.265305,-0.082683,-0.085964,-0.636864,-0.251601,-0.686736,-1.220570,-0.872972,-0.285522,-0.630555,...,-0.165527,-0.451885,-0.123157,-0.232351,-0.197695,-0.321001,-0.308827,-0.210567,-0.440283,-0.104728,-0.338442,-0.097507,-0.26872,-0.19559,-0.189446,-0.242677,-0.226082,-0.167025,-0.101344,-0.149435,-0.16783,-0.182647,-0.148962,-0.171994,-0.175678,-0.032054,-0.106901,-0.159448,-0.257318,-0.200138,-0.147242,-0.152215,-0.082305,-0.165547,-0.097026,-0.061946,-0.061946,-0.151059,-0.004979,-0.021419
98715,0.831926,-0.643452,-0.592187,1.196348,-0.308540,-0.103276,0.619740,-1.519691,0.659419,-1.601965,0.619842,0.622025,0.635027,0.667014,0.951494,0.649109,-0.887561,0.673167,1.375632,-0.368655,-0.261265,-0.280848,-0.676825,-0.387994,1.411724,-0.224626,-0.243328,-0.141667,-0.263995,-0.948090,-0.265305,-0.082683,-0.085964,-0.636864,-0.251601,-0.686736,-1.220570,1.153782,-0.285522,-0.630555,...,-0.165527,-0.451885,-0.123157,-0.232351,-0.197695,-0.321001,-0.308827,-0.210567,-0.440283,-0.104728,-0.338442,-0.097507,-0.26872,-0.19559,-0.189446,-0.242677,-0.226082,-0.167025,-0.101344,-0.149435,-0.16783,-0.182647,-0.148962,-0.171994,-0.175678,-0.032054,-0.106901,-0.159448,-0.257318,-0.200138,-0.147242,-0.152215,-0.082305,-0.165547,-0.097026,-0.061946,-0.061946,-0.151059,-0.004979,-0.021419
76064,-0.548379,-0.643452,-0.592187,-0.861957,-0.308540,-0.103276,0.480314,0.584587,0.5266

In [88]:
X_test_bowling

,Consistency,Form,U.A.E.,Scotland,Namibia,Nepal,Sri Lanka,Zimbabwe,New Zealand,England,West Indies,Bangladesh,Pakistan,Australia,Ireland,India,Kenya,South Africa,Netherlands,Hong Kong,ICC World XI,Africa XI,Canada,Bermuda,Afghanistan,P.N.G.,U.S.A.,Oman,Asia XI,Sharjah,ICCA Dubai,Al Amerat,Kirtipur,Karachi,Gujranwala,Lahore,Harare,Wellington,Aberdeen,Mumbai,...,Greater Noida,Nelson,Kwekwe,Kandy,Ayr,Ranchi,Dharamsala,Whangarei,Melbourne (Docklands),Schiedam,Toronto,Deventer,Georgetown,Nairobi (Jaff),Cairns,Chandigarh,Hyderabad (Sind),Lucknow,Port Moresby,Quetta,Derby,Hove,Leicester,Chelmsford,Jalandhar,Berri,Amritsar,Sialkot,Northampton,Worcester,Nairobi (Club),Moratuwa,New Delhi,Nairobi (Aga),Sargodha,New Plymouth,Ballarat,Patna,Devonport,Srinagar
113149,-0.614208,-0.643452,-0.592187,-0.011271,-0.30854,-0.103276,0.539451,0.640619,-1.588509,-1.601965,-0.433727,0.543203,-1.704035,-1.569296,0.881325,-0.365335,1.094094,-1.576499,1.304050,-0.368655,-0.261265,-0.280848,1.490280,-0.387994,-0.75048,-0.224626,-0.243328,-0.141667,-0.263995,-0.948090,-0.265305,-0.082683,-0.085964,-0.636864,-0.251601,-0.686736,-1.220570,-0.872972,-0.285522,-0.630555,...,-0.165527,-0.451885,-0.123157,-0.232351,-0.197695,-0.321001,-0.308827,-0.210567,-0.440283,-0.104728,3.002671,-0.097507,-0.268720,5.441683,-0.189446,-0.242677,-0.226082,-0.167025,-0.101344,-0.149435,-0.167830,-0.182647,-0.148962,-0.171994,-0.175678,-0.032054,-0.106901,-0.159448,-0.257318,-0.200138,-0.147242,-0.152215,-0.082305,-0.165547,-0.097026,-0.061946,-0.061946,-0.151059,-0.004979,-0.021419
68651,-0.539069,-0.643452,-0.592187,-0.861957,-0.30854,-0.103276,0.440247,0.546625,0.488474,0.490558,0.440729,-1.695187,0.457152,0.496953,-1.111364,0.476648,-0.887561,0.502091,1.215603,-0.368655,-0.261265,-0.280848,-0.676825,-0.387994,-0.75048,-0.224626,-0.243328,-0.141667,-0.263995,-0.948090,-0.265305,-0.082683,-0.085964,-0.636864,-0.251601,-0.686736,0.691365,-0.872972,-0.285522,-0.630555,...,-0.165527,-0.451885,-0.123157,-0.232351,-0.197695,-0.321001,-0.308827,-0.210567,-0.440283,-0.104728,-0.338442,-0.097507,-0.268720,-0.195590,-0.189446,-0.242677,-0.226082,-0.167025,-0.101344,-0.149435,-0.167830,-0.182647,-0.148962,-0.171994,-0.175678,-0.032054,-0.106901,-0.159448,-0.257318,-0.200138,-0.147242,-0.152215,-0.082305,-0.165547,-0.097026,-0.061946,-0.061946,-0.151059,-0.004979,-0.021419
27144,0.375341,-0.643452,1.735943,-0.861957,-0.30854,-0.103276,0.550027,0.650639,0.593026,0.595891,0.550277,0.553586,0.565943,0.600964,0.890568,0.582127,1.103286,-1.576499,1.313479,-0.368655,-0.261265,-0.280848,-0.676825,-0.387994,-0.75048,-0.224626,-0.243328,-0.141667,3.850088,1.017415,-0.265305,-0.082683,-0.085964,-0.636864,-0.251601,-0.686736,0.804073,1.092821,-0.285522,-0.630555,...,-0.165527,-0.451885,-0.123157,-0.232351,-0.197695,-0.321001,-0.308827,-0.210567,-0.440283,-0.104728,-0.338442,-0.097507,-0.268720,-0.195590,-0.189446,-0.242677,-0.226082,-0.167025,-0.101344,-0.149435,-0.167830,-0.182647,-0.148962,-0.171994,-0.175678,-0.032054,-0.106901,-0.159448,-0.257318,-0.200138,-0.147242,-0.152215,-0.082305,-0.165547,-0.097026,-0.061946,-0.061946,-0.151059,-0.004979,-0.021419
20480,1.299062,-0.643452,1.806797,-0.861957,-0.30854,-0.103276,0.619740,0.716691,0.659419,0.662781,0.619842,0.622025,0.635027,0.667014,-1.111364,-1.618767,1.163875,0.673167,1.375632,-0.368655,3.967480,3.335387,-0.676825,2.720038,-0.75048,-0.224626,-0.243328,-0.141667,-0.263995,1.078367,-0.265305,-0.082683,-0.085964,-0.636864,-0.251601,-0.686736,0.866859,1.153782,-0.285522,1.598259,...,-0.165527,-0.451885,-0.123157,-0.232351,-0.197695,-0.321001,-0.308827,-0.210567,-0.440283,-0.104728,3.133228,-0.097507,-0.268720,-0.195590,-0.189446,4.239506,-0.226082,-0.167025,-0.101344,-0.149435,-0.167830,5.122075,7.018778,-0.171994,-0.175678,-0.032054,9.450362,-0.159448,-0.257318,-0.200138,-0.147242,6.702085,-0.082305,-0.165547,-0.097026,-0.061946,-0.061946,-0.151059,-0.004979,-0.021419
56536,-1.192864,-0.643452,-0.592187,-0.861957,-0.30854,-0.103276,0.358781,

In [89]:
import numpy as np

X_train_bowling = np.array(X_train_bowling)
X_test_bowling = np.array(X_test_bowling)
y_train_bowling = np.array(y_train_bowling)
y_test_bowling = np.array(y_test_bowling)

In [90]:
X_train_bowling

array([[ 1.21260447e+00, -6.43451700e-01, -5.92187304e-01, ...,
         6.75473184e+00, -4.97888452e-03, -2.14192603e-02],
       [ 1.26713919e+00, -6.43451700e-01,  1.78155103e+00, ...,
        -1.51059479e-01, -4.97888452e-03, -2.14192603e-02],
       [ 1.04717039e+00, -6.43451700e-01, -5.92187304e-01, ...,
        -1.51059479e-01, -4.97888452e-03, -2.14192603e-02],
       ...,
       [ 5.53156178e-02,  1.54735669e+00, -5.92187304e-01, ...,
        -1.51059479e-01, -4.97888452e-03, -2.14192603e-02],
       [-5.54386367e-02,  1.72662350e+00, -5.92187304e-01, ...,
        -1.51059479e-01, -4.97888452e-03, -2.14192603e-02],
       [-1.09299848e+00, -6.43451700e-01, -5.92187304e-01, ...,
        -1.51059479e-01, -4.97888452e-03, -2.14192603e-02]])

# Model Building

In [91]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

In [92]:
models_bowling = []
models_bowling.append(('DTC', DecisionTreeClassifier(criterion= 'entropy', random_state=0)))
models_bowling.append(('KNC', KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)))
models_bowling.append(('NB', GaussianNB()))
models_bowling.append(('RFC', RandomForestClassifier(n_estimators=500, criterion='entropy', random_state=0)))
models_bowling.append(('SVC', SVC(random_state = 0, kernel = 'rbf')))
models_bowling.append(('XGB', XGBClassifier()))

In [ ]:
results = []
names = []
kFold = []

for name, model in models_bowling:
  model.fit(X_train_bowling, y_train_bowling)
  y_pred = model.predict(X_test_bowling)
  accuracies = accuracy_score(y_test_bowling, y_pred)
  fold = cross_val_score(estimator = model, X = X_train_bowling, y = y_train_bowling, cv = 10)
  results.append(accuracies*100)
  names.append(name)
  kFold.append(fold.mean()*100)
final_comparison_bowling = pd.DataFrame(list(zip(names, results, kFold)), columns = ['Model Name', 'Accuracy', 'K-Fold'])

In [ ]:
final_comparison_bowling.sort_values(by=['Accuracy'], ascending=False)